这部分主要是由于 python 顺序执行 , 一些主要函数的辅助函数必须放到主要函数前面导致其出现时可能不知道要干什么 , 但先往下溯源到应用再回头看实现即可

整体上的流程 : 

- 配置 tokenizer , 模型 , dataset
- 设置训练与评估流程
- 运行 

`yield` 是干什么的 ? 

它主要是相当于一个函数级别的 iterator :
- 被调用
- 返回当下 iterator 值 
- 步进 
- 当下次被再调用时 , 不会从零开始 ( 这里对比 `return` )

为什么要这样 : 这种流式的数据能优化性能 , 防止爆内存 . 而其应用的句子本身只是用来做一个统计与映射 , 没有过高的并行化计算要求 .

如果用 return , 就需要一次性以 list ( 或类似东西 ) 将全体数据加载到内存里 :

```python
def get_all_sentences(ds, lang):
    sentences = []
    for item in ds:
        sentences.append(item['translation'][lang])
    return sentences
```

后面 hugging face 提供的 `train_from_iterator` 可以对接这种数据传输方式 , 具体如何实现不是这里的重点 .

而可以发现只在这里用到了 这个 yield 与 iterator 机制 , 后面 train 时反而没有用 , 这是因为 Dataset 有自己的优化的读取方式 .

In [ ]:
def get_all_sentences(ds, lang):
    for item in ds:
        yield item['translation'][lang]

这里对 tokenizer 的使用更多是照抄 huggingface 给的代码 , 因为整体步骤较为常见了 . 

整体代码的作用是构建一个从文本到 tokenizer ( 而非 tokens , 这一过程还需要加入特殊字符 ( 如 `[SOS] [PAD]` 等 , 在构建 Dataset 时才会去完成 ) ) 

这里的 tokenizer 在用语上也说是 " 训练得到一个 tokenizer " , 但与模型训练的训练不是一个概念 . tokenizer 的 " 训练 " 是一种统计学习 , 主要步骤有 : 遍历整个语料库 , 统计每个词出现的次数 , 过滤掉出现频率 < 2 的词 , 构建词典 等 . 其需要训练也主要是因为不同语料库的词频统计规律不同 ( 比如不同语言用的字典显然不同 , 一个魔幻小说用到的词用到一个论文里也会出现大量 `[UNK]` )

这里的 tokenizer_path 能直接 format 是因为 config 文件中已经用了 `{0}` 占好位了 . 

一个 tokenizer 的配置 , 可以看到大概能找到三部分 : tokenizer 本身的原则 , pre_tokenizer 与 trainer

Token 最基础的显然是单子划分原则 . 这里使用的 WordLevel 是最基础的一种 , 顾名思义 , 直接单个词记一个 token , 更复杂的划分方法当然还有很多 .

pre_tokenizer 主要是对文本进行划分 : 这里的 Whitespace 相当于用空格进行划分 

Trainer 则是指定需要插入的特殊字符 , 以及不被识别为 `[UNK]` 所需的最小频率等 .

组合完成后就能训练并保存 , 以后就能直接调用了 .

显然 tokenizer 是模块分离组合的产物 , 但为什么是这样的三部分分割设计 ( 比如为什么 pre_tokenizer 要单分出来之类的 ) , 我也不明白 , 还得多积累 ( 

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

from pathlib import Path

In [ ]:
def get_or_build_tokenizer(config, ds, lang):
    tokenizer_path = Path(config['tokenizer_file'].format(lang))
    if not Path.exists(tokenizer_path):
        tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
        tokenizer.pre_tokenizer = Whitespace()
        trainer = WordLevelTrainer(special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"], min_frequency=2)
        tokenizer.train_from_iterator(get_all_sentences(ds, lang), trainer=trainer)
        tokenizer.save(str(tokenizer_path))
    else:
        tokenizer = Tokenizer.from_file(str(tokenizer_path))
    return tokenizer

接下来是数据集获取

更准确来讲它还会承接将上面的 tokenzier 具体化的工作 ( tokenizer 不仅仅是参与词转 id , 还要在推理时参与 id 转词 )

而切分步骤是在生成 Dataset 之前进行的 , 这个其实比较显然 . 而最关键的 Dataset 构造可以在另一个文件中看到 . 

max_len 是为了保证后续所有输入句子长度相同 , 需要到时候补 `[PAD]` . 而模型也会手动在 config 里设置超参数 seq_len , 根据这里的 max_len 选择 , 从而将一些过长的句子截断防止每个句子都有过长的 padding

最终获得 DataLoader

In [ ]:
from datasets import load_dataset
from torch.utils.data import DataLoader, random_split
from dataset import BilingualDataset
from model import build_transformer 

In [ ]:
def get_ds(config):

    ds_raw = load_dataset(f"{config['datasource']}", f"{config['lang_src']}-{config['lang_tgt']}", split='train')

    # Build tokenizers
    tokenizer_src = get_or_build_tokenizer(config, ds_raw, config['lang_src'])
    tokenizer_tgt = get_or_build_tokenizer(config, ds_raw, config['lang_tgt'])

    train_ds_size = int(0.9 * len(ds_raw))
    val_ds_size = len(ds_raw) - train_ds_size
    train_ds_raw, val_ds_raw = random_split(ds_raw, [train_ds_size, val_ds_size])

    train_ds = BilingualDataset(train_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])
    val_ds = BilingualDataset(val_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])

    max_len_src = 0
    max_len_tgt = 0

    for item in ds_raw:
        src_ids = tokenizer_src.encode(item['translation'][config['lang_src']]).ids
        tgt_ids = tokenizer_tgt.encode(item['translation'][config['lang_tgt']]).ids
        max_len_src = max(max_len_src, len(src_ids))
        max_len_tgt = max(max_len_tgt, len(tgt_ids))

    print(f'Max length of source sentence: {max_len_src}')
    print(f'Max length of target sentence: {max_len_tgt}')
    

    train_dataloader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True)
    val_dataloader = DataLoader(val_ds, batch_size=1, shuffle=True)

    return train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt

获取模型反而是最好理解的 , 毕竟其文件本身是自己写的

In [ ]:
def get_model(config, vocab_src_len, vocab_tgt_len):
    model = build_transformer(vocab_src_len, vocab_tgt_len, config["seq_len"], config['seq_len'], d_model=config['d_model'])
    return model

接下来这一部分是验证部分 . 其实可以最后再关注这部分而直接去看 train 函数的部分 , 最后再在函数里插入这里的验证模块而已 .

首先对于单个待预测句子 , 选用贪心的策略 , 每次都选当下可能最高的那个 . 

处理上 , 首先将原句子 encode 好 , 扔到 encoder 里 , 再让目标句子从 SOS 开始生成 . 因为这里应用的是翻译任务 , 所以生成的本身也是一个完整句子 ( 而非某个从中间突然开始而缺少原句作为开头的句子 )

这是 encoder-decoder 混合架构与单 decoder 架构的一个非常大的区别 . 如果是 decoder 任务 , 就会得到 `[[SOS] , en-sentence , [SEP] , it-sentence]` 这样的生成结构

搞清楚到底哪里该输入什么 , 剩下的就简单了 : 将原句子放入 encoder , 让 decoder 一直生成 , 直到其生成 `[EOS]` 或最大长度到达原设定句子最大长度 . 

最后再压缩成一个向量尺寸的输出句子 .

In [ ]:
import torch
from dataset import causal_mask

In [ ]:
def greedy_decode(model, source, source_mask, tokenizer_src, tokenizer_tgt, max_len, device):
    sos_idx = tokenizer_tgt.token_to_id('[SOS]')
    eos_idx = tokenizer_tgt.token_to_id('[EOS]')

    encoder_output = model.encode(source, source_mask)

    decoder_input = torch.empty(1, 1).fill_(sos_idx).type_as(source).to(device)

    while True:
        if decoder_input.size(1) == max_len:
            break

        decoder_mask = causal_mask(decoder_input.size(1)).type_as(source_mask).to(device)

        out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)

        prob = model.project(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        decoder_input = torch.cat(
            [decoder_input, torch.empty(1, 1).type_as(source).fill_(next_word.item()).to(device)], dim=1
        )

        if next_word == eos_idx:
            break

    return decoder_input.squeeze(0)

In [ ]:
import os
import torchmetrics

以下是预测函数 

发现真的有用的部分就是 with torch.no_grad() 以后的部分 :

每从验证集中取一个 batch , 取单个句子进行预测 , 预测结果与原结果进行展示 . 而显然翻译没法轻易得到 acc , 这里更多是展示翻译效果的作用 . 

里面会涉及到一些控制台调试之类的东西 , 与主线无关的就略去了 . writer 作为一个数据记录功能的加强 , if writer 部分代码也因与主线无关而略掉 . 

In [ ]:
def run_validation(model, validation_ds, tokenizer_src, tokenizer_tgt, max_len, device, print_msg, global_step, writer, num_examples=2):
    model.eval()
    count = 0

    source_texts = []
    expected = []
    predicted = []

    try:
        with os.popen('stty size', 'r') as console:
            _, console_width = console.read().split()
            console_width = int(console_width)
    except:
        console_width = 80

    with torch.no_grad():
        for batch in validation_ds:
            count += 1
            encoder_input = batch["encoder_input"].to(device) 
            encoder_mask = batch["encoder_mask"].to(device) 

            assert encoder_input.size(0) == 1, "Batch size must be 1 for validation"

            model_out = greedy_decode(model, encoder_input, encoder_mask, tokenizer_src, tokenizer_tgt, max_len, device)

            source_text = batch["src_text"][0]
            target_text = batch["tgt_text"][0]
            model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy())

            source_texts.append(source_text)
            expected.append(target_text)
            predicted.append(model_out_text)
            
            # Print the source, target and model output
            print_msg('-'*console_width)
            print_msg(f"{f'SOURCE: ':>12}{source_text}")
            print_msg(f"{f'TARGET: ':>12}{target_text}")
            print_msg(f"{f'PREDICTED: ':>12}{model_out_text}")

            if count == num_examples:
                print_msg('-'*console_width)
                break
    
    if writer:
        metric = torchmetrics.CharErrorRate()
        cer = metric(predicted, expected)
        writer.add_scalar('validation cer', cer, global_step)
        writer.flush()

        metric = torchmetrics.WordErrorRate()
        wer = metric(predicted, expected)
        writer.add_scalar('validation wer', wer, global_step)
        writer.flush()

        metric = torchmetrics.BLEUScore()
        bleu = metric(predicted, expected)
        writer.add_scalar('validation BLEU', bleu, global_step)
        writer.flush()


In [ ]:
from torch.utils.tensorboard import SummaryWriter
from config import get_weights_file_path, latest_weights_file_path
import torch.nn as nn
from tqdm import tqdm


SummaryWriter 是一个记录版性质的东西 , 不影响核心 . 本质上删去这些记录功能 , 仅靠 torch 就能训练了 .

整体上仍然是基本的模型训练结构 , 如果前面都能看懂的话这里就会非常容易 , 需要注意的更多是何时配置什么 , 使用什么之类的细节 .

epoch 是指遍历一整个数据集 , step 是指经过的 batch 的数量 .

padding 并非作为特殊字符参与训练 , 而是作为一个标志让模型的任何一部分都忽略掉他 , 比如 mask 的生成 , 比如交叉熵能设置忽略 padding . 而同行内对交叉熵用 label_smoothing 会将最大值的一部分概率分给别的 token , 来提升一定的表现 . 

注意 : 数据 , 模型 , loss 等是都要用 `.to(device)` 挪到 cuda 上的 .

计算 loss 时 , 首先经过了展平操作 , 来将输入与 label 变成最简单的两个向量 . 

而验证与保存参数是每 epoch 为单位进行的 ( 而非每 step ) .

In [ ]:
def train_model(config):

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Using device:", device)
    device = torch.device(device)

    Path(f"{config['datasource']}_{config['model_folder']}").mkdir(parents=True, exist_ok=True)

    train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
    model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

    writer = SummaryWriter(config['experiment_name'])

    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], eps=1e-9)

    initial_epoch = 0
    global_step = 0
    preload = config['preload']
    model_filename = latest_weights_file_path(config) if preload == 'latest' else get_weights_file_path(config, preload) if preload else None
    if model_filename:
        print(f'Preloading model {model_filename}')
        state = torch.load(model_filename)
        model.load_state_dict(state['model_state_dict'])
        initial_epoch = state['epoch'] + 1
        optimizer.load_state_dict(state['optimizer_state_dict'])
        global_step = state['global_step']
    else:
        print('No model to preload, starting from scratch')

    loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1).to(device)

    for epoch in range(initial_epoch, config['num_epochs']):
        torch.cuda.empty_cache()
        model.train()
        batch_iterator = tqdm(train_dataloader, desc=f"Processing Epoch {epoch:02d}")
        for batch in batch_iterator:

            encoder_input = batch['encoder_input'].to(device) 
            decoder_input = batch['decoder_input'].to(device) 
            encoder_mask = batch['encoder_mask'].to(device) 
            decoder_mask = batch['decoder_mask'].to(device) 

            encoder_output = model.encode(encoder_input, encoder_mask) 
            decoder_output = model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask) 
            proj_output = model.project(decoder_output) 

            label = batch['label'].to(device) 

            loss = loss_fn(proj_output.view(-1, tokenizer_tgt.get_vocab_size()), label.view(-1))
            batch_iterator.set_postfix({"loss": f"{loss.item():6.3f}"})

            writer.add_scalar('train loss', loss.item(), global_step)
            writer.flush()

            loss.backward()

            optimizer.step()
            optimizer.zero_grad(set_to_none=True)

            global_step += 1

        run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config['seq_len'], device, lambda msg: batch_iterator.write(msg), global_step, writer)

        model_filename = get_weights_file_path(config, f"{epoch:02d}")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'global_step': global_step
        }, model_filename)

In [ ]:
import warnings
from config import get_config

In [ ]:
if __name__ == '__main__':
    warnings.filterwarnings("ignore")
    config = get_config()
    train_model(config)

## 补充 

### 总的 import 列表
```python
from model import build_transformer 
from dataset import BilingualDataset, causal_mask 
from config import get_config, get_weights_file_path, latest_weights_file_path

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split 

import warnings
from tqdm import tqdm
import os
from pathlib import Path 

from datasets import load_dataset 
from tokenizers import Tokenizer 
from tokenizers.models import WordLevel 
from tokenizers.trainers import WordLevelTrainer 
from tokenizers.pre_tokenizers import Whitespace 

import torchmetrics
from torch.utils.tensorboard import SummaryWriter
```
